# Man-in-the-Middle (MitM) Attacks - Hands-On Lab

**Part of HackLearn Pro - Module #17**

This interactive notebook provides practical exercises for understanding, detecting, and defending against Man-in-the-Middle attacks. All examples are for educational purposes and should only be used in authorized testing environments.

## Learning Objectives
- Implement TLS certificate validation to detect MitM attacks
- Monitor ARP tables for spoofing attacks
- Detect SSL stripping and downgrade attacks
- Query Certificate Transparency logs
- Configure HSTS for MitM prevention

---

## Setup & Dependencies

Install required Python packages for network security analysis.

In [ ]:
# Install required packages
!pip install certifi requests scapy

print("[+] All dependencies installed successfully!")

---

## Lab 1: TLS Certificate Validation

Learn to validate TLS certificates and detect self-signed, expired, or fraudulent certificates that indicate potential MitM attacks.

In [ ]:
import ssl
import socket
import certifi
from datetime import datetime

def check_certificate(hostname, port=443):
    """Validate TLS certificate and detect potential MitM"""
    context = ssl.create_default_context(cafile=certifi.where())
    
    try:
        with socket.create_connection((hostname, port), timeout=10) as sock:
            with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                cert = ssock.getpeercert()
                
                # Validate expiration
                not_after = datetime.strptime(cert['notAfter'], '%b %d %H:%M:%S %Y %Z')
                days_until_expiry = (not_after - datetime.now()).days
                
                print(f"[+] {hostname}: VALID certificate")
                print(f"    Issued to: {dict(x[0] for x in cert['subject'])}")
                print(f"    Issued by: {dict(x[0] for x in cert['issuer'])}")
                print(f"    Valid until: {cert['notAfter']} ({days_until_expiry} days)")
                print(f"    Serial: {cert['serialNumber']}")
                
                # Check Subject Alternative Names
                if 'subjectAltName' in cert:
                    san = [name[1] for name in cert['subjectAltName']]
                    print(f"    SAN: {', '.join(san[:5])}...")  # Show first 5
                
                # Warnings
                if days_until_expiry < 30:
                    print(f"    [!] WARNING: Certificate expires in {days_until_expiry} days")
                
                return True
                
    except ssl.SSLCertVerificationError as e:
        print(f"[-] {hostname}: CERTIFICATE VALIDATION FAILED")
        print(f"    Error: {e}")
        print(f"    [!] POSSIBLE MAN-IN-THE-MIDDLE ATTACK!")
        return False
    except ssl.SSLError as e:
        print(f"[-] {hostname}: SSL ERROR - {e}")
        return False
    except Exception as e:
        print(f"[-] {hostname}: CONNECTION FAILED - {e}")
        return False

# Test legitimate sites
print("Testing TLS Certificate Validation:\n")
sites = ['google.com', 'github.com', 'microsoft.com']

for site in sites:
    check_certificate(site)
    print()

### Exercise 1.1: Certificate Pinning

Implement certificate pinning by storing SHA-256 fingerprints and validating future connections.

In [ ]:
import hashlib

def get_certificate_fingerprint(hostname, port=443):
    """Extract SHA-256 fingerprint of certificate"""
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE  # Don't validate (we're just getting fingerprint)
    
    with socket.create_connection((hostname, port)) as sock:
        with context.wrap_socket(sock, server_hostname=hostname) as ssock:
            der_cert = ssock.getpeercert(binary_form=True)
            fingerprint = hashlib.sha256(der_cert).hexdigest()
            return fingerprint

def validate_pinned_certificate(hostname, expected_fingerprint, port=443):
    """Validate certificate matches pinned fingerprint"""
    actual_fingerprint = get_certificate_fingerprint(hostname, port)
    
    if actual_fingerprint == expected_fingerprint:
        print(f"[+] {hostname}: Certificate matches pinned fingerprint")
        print(f"    SHA-256: {actual_fingerprint}")
        return True
    else:
        print(f"[-] {hostname}: CERTIFICATE PINNING FAILURE!")
        print(f"    Expected: {expected_fingerprint}")
        print(f"    Actual:   {actual_fingerprint}")
        print(f"    [!] POSSIBLE MAN-IN-THE-MIDDLE ATTACK!")
        return False

# Pin Google's certificate (in production, store this securely)
google_fingerprint = get_certificate_fingerprint('google.com')
print(f"Google certificate SHA-256 fingerprint:\n{google_fingerprint}\n")

# Later validation
validate_pinned_certificate('google.com', google_fingerprint)

---

## Lab 2: ARP Spoofing Detection

Monitor ARP tables for suspicious changes indicating ARP poisoning attacks. This requires admin/root privileges to capture packets.

**Note:** This lab requires running as administrator/root and may need to be executed outside Jupyter.

In [ ]:
from scapy.all import ARP, sniff
from collections import defaultdict
import time

# Track IP -> MAC mappings
arp_table = defaultdict(set)

def detect_arp_spoofing(packet):
    """Detect ARP poisoning by monitoring for duplicate IP/MAC pairs"""
    if ARP in packet and packet[ARP].op == 2:  # ARP reply
        ip = packet[ARP].psrc
        mac = packet[ARP].hwsrc
        
        # Check if we've seen this IP with a different MAC
        if ip in arp_table and mac not in arp_table[ip]:
            print(f"\n[!] ARP SPOOFING DETECTED!")
            print(f"    IP Address: {ip}")
            print(f"    Previous MAC(s): {', '.join(arp_table[ip])}")
            print(f"    New MAC: {mac}")
            print(f"    [!] Possible attacker MAC: {mac}")
            
            # Log to file
            with open('arp_spoofing_log.txt', 'a') as f:
                timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
                f.write(f"{timestamp} - ARP spoofing: {ip} changed from {arp_table[ip]} to {mac}\n")
        
        # Add to tracking table
        arp_table[ip].add(mac)
        print(f"[*] ARP: {ip} <-> {mac}")

print("[*] ARP Spoofing Detection")
print("[*] This requires admin/root privileges")
print("[*] If running in Jupyter, you may need to run this in a separate Python script\n")

# Uncomment to run (requires root/admin)
# print("[*] Starting ARP spoofing detection... (Press Ctrl+C to stop)\n")
# try:
#     sniff(filter="arp", prn=detect_arp_spoofing, store=0, count=20)
# except KeyboardInterrupt:
#     print("\n[*] Stopping detection...")
#     print(f"[*] Monitored {len(arp_table)} unique IP addresses")

---

## Lab 3: Certificate Transparency Monitoring

Monitor Certificate Transparency logs to detect rogue certificates issued for your domains.

In [ ]:
import requests
import json
from datetime import datetime

def monitor_certificate_transparency(domain):
    """Query CT logs for certificates issued to domain"""
    url = f"https://crt.sh/?q=%.{domain}&output=json"
    
    try:
        print(f"[*] Querying Certificate Transparency logs for: {domain}")
        print(f"[*] This may take 10-30 seconds...\n")
        
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            certificates = response.json()
            
            print(f"[+] Found {len(certificates)} certificates for *.{domain}\n")
            
            # Group by issuer
            issuers = {}
            for cert in certificates:
                issuer = cert.get('issuer_name', 'Unknown')
                if issuer not in issuers:
                    issuers[issuer] = []
                issuers[issuer].append(cert)
            
            # Display summary
            print("Certificate Issuers:")
            for issuer, certs in list(issuers.items())[:10]:  # Show top 10
                print(f"\n{issuer}")
                print(f"  Certificates: {len(certs)}")
                
                # Show recent certificates
                recent = sorted(certs, key=lambda x: x.get('entry_timestamp', ''), reverse=True)[:2]
                for cert in recent:
                    cn = cert.get('common_name', cert.get('name_value', 'N/A'))
                    timestamp = cert.get('entry_timestamp', 'Unknown')
                    print(f"    - {cn}")
                    print(f"      Logged: {timestamp[:19] if len(timestamp) > 19 else timestamp}")
            
            # Alert on suspicious issuers
            suspicious_keywords = ['free', 'test', 'fake', 'phishing']
            suspicious_found = False
            
            for cert in certificates:
                issuer_lower = cert.get('issuer_name', '').lower()
                if any(keyword in issuer_lower for keyword in suspicious_keywords):
                    if not suspicious_found:
                        print("\n[!] SECURITY ALERTS:\n")
                        suspicious_found = True
                    
                    print(f"[!] SUSPICIOUS CERTIFICATE DETECTED!")
                    print(f"    Domain: {cert.get('common_name', 'N/A')}")
                    print(f"    Issuer: {cert.get('issuer_name', 'N/A')}")
                    print(f"    Logged: {cert.get('entry_timestamp', 'Unknown')}")
                    print(f"    [!] Possible rogue certificate!\n")
            
            if not suspicious_found:
                print("\n[+] No suspicious certificates detected")
            
            # Show unique subdomains
            print("\nDiscovered subdomains (first 20):")
            unique_domains = set()
            for cert in certificates:
                name = cert.get('name_value', '')
                for subdomain in name.split('\n'):
                    unique_domains.add(subdomain.strip())
            
            for subdomain in list(sorted(unique_domains))[:20]:
                print(f"    - {subdomain}")
            
            if len(unique_domains) > 20:
                print(f"    ... and {len(unique_domains) - 20} more")
            
            return certificates
            
    except Exception as e:
        print(f"[-] Error querying CT logs: {e}")
        return []

# Monitor example domains (replace with your own)
print("Certificate Transparency Monitoring\n")
print("Replace 'example.com' with your own domain to monitor for rogue certificates\n")
print("="*60)

# Test with a public domain
certificates = monitor_certificate_transparency('github.com')

print("\n" + "="*60)
print(f"\n[+] Analysis complete. Found {len(certificates)} total certificates.")

---

## Lab 4: HSTS (HTTP Strict Transport Security) Validation

Check if websites properly implement HSTS to prevent SSL stripping attacks.

In [ ]:
import requests

def check_hsts(url):
    """Check if website implements HSTS properly"""
    if not url.startswith('https://'):
        url = 'https://' + url.replace('http://', '')
    
    try:
        print(f"[*] Checking HSTS for: {url}")
        response = requests.get(url, timeout=10)
        
        # Check for HSTS header
        hsts_header = response.headers.get('Strict-Transport-Security', None)
        
        if hsts_header:
            print(f"[+] HSTS ENABLED")
            print(f"    Header: {hsts_header}")
            
            # Parse HSTS directives
            directives = {}
            for directive in hsts_header.split(';'):
                directive = directive.strip()
                if '=' in directive:
                    key, value = directive.split('=', 1)
                    directives[key.strip()] = value.strip()
                else:
                    directives[directive] = True
            
            # Check max-age
            max_age = int(directives.get('max-age', 0))
            if max_age >= 31536000:  # 1 year
                print(f"    [+] max-age: {max_age} seconds ({max_age // 31536000} years) - GOOD")
            elif max_age > 0:
                print(f"    [!] max-age: {max_age} seconds ({max_age // 86400} days) - TOO SHORT")
                print(f"        Recommended: At least 31536000 (1 year)")
            
            # Check includeSubDomains
            if 'includeSubDomains' in directives:
                print(f"    [+] includeSubDomains: YES - GOOD")
            else:
                print(f"    [!] includeSubDomains: NO - MISSING")
                print(f"        Recommended: Include this directive")
            
            # Check preload
            if 'preload' in directives:
                print(f"    [+] preload: YES - EXCELLENT")
            else:
                print(f"    [-] preload: NO - OPTIONAL (submit to hstspreload.org)")
            
            # Overall assessment
            if max_age >= 31536000 and 'includeSubDomains' in directives:
                print(f"\n    [+] OVERALL: STRONG HSTS CONFIGURATION")
            else:
                print(f"\n    [!] OVERALL: HSTS NEEDS IMPROVEMENT")
        
        else:
            print(f"[-] HSTS NOT ENABLED")
            print(f"    [!] Site vulnerable to SSL stripping attacks")
            print(f"    [!] Recommendation: Add HSTS header")
            print(f"        Strict-Transport-Security: max-age=31536000; includeSubDomains; preload")
        
        print()
        return hsts_header is not None
        
    except Exception as e:
        print(f"[-] Error checking HSTS: {e}\n")
        return False

# Test multiple websites
print("HSTS Implementation Analysis\n")
print("="*60)

test_sites = [
    'https://google.com',
    'https://github.com',
    'https://microsoft.com',
    'https://bankofamerica.com'
]

results = []
for site in test_sites:
    has_hsts = check_hsts(site)
    results.append((site, has_hsts))
    print("-" * 60)

# Summary
print("\nSummary:")
enabled_count = sum(1 for _, hsts in results if hsts)
print(f"HSTS Enabled: {enabled_count}/{len(results)} sites")
print(f"HSTS Disabled: {len(results) - enabled_count}/{len(results)} sites")

---

## Lab 5: Security Headers Analysis

Analyze all security headers to identify potential vulnerabilities to MitM and other attacks.

In [ ]:
def analyze_security_headers(url):
    """Comprehensive security header analysis"""
    if not url.startswith('http'):
        url = 'https://' + url
    
    try:
        print(f"[*] Analyzing security headers for: {url}\n")
        response = requests.get(url, timeout=10, allow_redirects=True)
        
        # Security headers to check
        security_headers = {
            'Strict-Transport-Security': {
                'purpose': 'Prevents SSL stripping attacks',
                'severity': 'HIGH'
            },
            'Content-Security-Policy': {
                'purpose': 'Prevents XSS and injection attacks',
                'severity': 'HIGH'
            },
            'X-Frame-Options': {
                'purpose': 'Prevents clickjacking attacks',
                'severity': 'MEDIUM'
            },
            'X-Content-Type-Options': {
                'purpose': 'Prevents MIME-sniffing attacks',
                'severity': 'MEDIUM'
            },
            'Referrer-Policy': {
                'purpose': 'Controls referrer information leakage',
                'severity': 'LOW'
            },
            'Permissions-Policy': {
                'purpose': 'Controls browser feature access',
                'severity': 'LOW'
            }
        }
        
        print("Security Headers Analysis:")
        print("="*60)
        
        present = []
        missing = []
        
        for header, info in security_headers.items():
            value = response.headers.get(header)
            
            if value:
                present.append(header)
                print(f"[+] {header}: PRESENT")
                print(f"    Value: {value[:80]}{'...' if len(value) > 80 else ''}")
                print(f"    Purpose: {info['purpose']}")
            else:
                missing.append(header)
                print(f"[-] {header}: MISSING ({info['severity']} severity)")
                print(f"    Purpose: {info['purpose']}")
                print(f"    [!] Recommendation: Implement this header")
            
            print()
        
        # Overall security score
        score = (len(present) / len(security_headers)) * 100
        
        print("="*60)
        print(f"\nSecurity Score: {score:.1f}%")
        print(f"Headers Present: {len(present)}/{len(security_headers)}")
        print(f"Headers Missing: {len(missing)}/{len(security_headers)}")
        
        if score >= 80:
            print("\n[+] OVERALL ASSESSMENT: EXCELLENT")
        elif score >= 60:
            print("\n[!] OVERALL ASSESSMENT: GOOD (but needs improvement)")
        elif score >= 40:
            print("\n[!] OVERALL ASSESSMENT: FAIR (significant gaps)")
        else:
            print("\n[-] OVERALL ASSESSMENT: POOR (critical security gaps)")
        
        return present, missing
        
    except Exception as e:
        print(f"[-] Error analyzing headers: {e}")
        return [], []

# Analyze a website
analyze_security_headers('https://github.com')

---

## Summary & Key Takeaways

### What We Learned

1. **TLS Certificate Validation**
   - Verify certificate chains to detect fraudulent certificates
   - Implement certificate pinning for critical applications
   - Monitor expiration dates and Subject Alternative Names

2. **Certificate Transparency**
   - Monitor CT logs to detect rogue certificates for your domains
   - Essential defense against compromised CAs
   - Provides transparency into global certificate issuance

3. **HSTS Implementation**
   - Primary defense against SSL stripping attacks
   - Requires max-age ≥ 31536000 (1 year)
   - Include `includeSubDomains` and `preload` directives
   - Submit to HSTS preload list for browser enforcement

4. **Security Headers**
   - Multiple layers of defense beyond HSTS
   - CSP prevents injection attacks
   - X-Frame-Options prevents clickjacking
   - Defense-in-depth approach essential

### Defense Checklist

- ✅ Implement HSTS with long max-age (1+ years)
- ✅ Enable certificate pinning for critical apps
- ✅ Monitor Certificate Transparency logs
- ✅ Use TLS 1.3 exclusively (disable TLS 1.0/1.1)
- ✅ Deploy comprehensive security headers
- ✅ Use VPNs on untrusted networks
- ✅ Validate certificates programmatically
- ✅ Monitor network traffic for anomalies

### Additional Resources

- **OWASP Transport Layer Protection:** https://cheatsheetseries.owasp.org/cheatsheets/Transport_Layer_Protection_Cheat_Sheet.html
- **HSTS Preload List:** https://hstspreload.org/
- **Certificate Transparency:** https://certificate.transparency.dev/
- **Mozilla SSL Configuration Generator:** https://ssl-config.mozilla.org/

---

## Legal & Ethical Reminder

All MitM attack techniques must only be used:
- On systems you own
- With explicit written authorization for penetration testing
- In isolated lab environments for educational purposes

Unauthorized interception of communications violates:
- Computer Fraud and Abuse Act (CFAA), 18 U.S.C. § 1030
- Wiretap Act, 18 U.S.C. § 2511

**Maximum penalties:** $250,000 fine + 20 years imprisonment

---

**Congratulations on completing the Man-in-the-Middle Attacks lab!**